In [1]:
import pandas as pd
import numpy as np
import tarfile

In [2]:
file_name = "./breast_3d_ultrasound_lowres.tar.gz"
data_dir = './Data' # uncompressed data is here

# with tarfile.open(file_name, "r") as f:
#     f.extractall(data_dir)
# os.remove(file_name)

In [4]:
df = pd.read_csv('./Data/trainingSet.csv')
df

,patientID,studyID,scanName,studyDate,annotationLowResolution1,annotationLowResolution2,annotationLowResolution3,spacingLowResolution1,spacingLowResolution2,spacingLowResolution3,annotationHighResolution1,annotationHighResolution2,annotationHighResolution3,spacingHighResolution1,spacingHighResolution2,spacingHighResolution3,histology
0,2000801,st14,al,20110615,142,101,21,0.6,0.6,0.6,404,179,203,0.210762,0.070254,0.525000,21
1,2000801,st14,ll,20110615,108,66,21,0.6,0.6,0.6,307,179,243,0.210762,0.070254,0.525000,21
2,2000801,st19,al,20130125,160,126,18,0.6,0.6,0.6,445,100,174,0.215602,0.107801,0.525000,21
3,2002243,st01,mr,20130410,171,140,24,0.6,0.6,0.6,476,134,158,0.215602,0.107801,0.525000,2
4,2002287,st07,al,20140910,180,164,12,0.6,0.6,0.6,501,67,131,0.215602,0.107801,0.525000,2
5,2002117,st01,al,20121017,136,84,24,0.6,0.6,0.6,387,205,222,0.210762,0.070254,0.525000,21
6,2002117,st01,ll,20121017,38,102,36,0.6,0.6,0.6,108,307,201,0.210762,0.070254,0.525000,21
7,2002117,st01,ml,20121017,219,114,30,0.6,0.6,0.6,623,256,188,0.210762,0.070254,0.525000,21
8,2004445,st01,ll,20150109,179,205,12,0.6,0.6,0.6,502,67,84,0.213988,0.106994,0.525000,2
9,2001283,st01,lr,20110601,181,191,23,0.6,0.6,0.6,511,195,100,0.212375,0.070792,0.525000,2


In [44]:
def make_folds(data, folds):
    def KFold(data, folds = 5):
        result = [data[i::folds] for i in range(folds)]
        np.random.shuffle(result)
        return result
    
    data = data.assign(breast = data['scanName'].str[1])
    
    unique_data = data.drop_duplicates(subset=["patientID", "studyDate", "breast"], keep=False)
    no_unique = data.drop(unique_data.index.values.tolist())
    first_duplicates = no_unique.drop_duplicates(subset=["patientID", "studyDate", "breast"], keep='first')
    no_first = no_unique.drop(first_duplicates.index.values.tolist())
        
    final_data = pd.concat([unique_data, first_duplicates])
    final_folds = KFold(final_data, folds)

    for i in range(len(final_folds)):
        for idx, row in no_first.iterrows():
            if row['patientID'] in final_folds[i]['patientID'].values:
                final_folds[i] = final_folds[i].append(row)
    
    final_idx = [fold_i.index.tolist() for fold_i in final_folds]
    
    # 2 -> 0, 20 -> 1, 21 -> 2  
    final_lbl = []
    for i in range(len(final_folds)):
        lbl = []
        for idx, row in final_folds[i].iterrows():
            if row['histology'] == 2:
                lbl.append(0)
            elif row['histology'] == 20:
                lbl.append(1)
            elif row['histology'] == 21:
                lbl.append(2)
        final_lbl.append(lbl)
        
    return final_folds, final_idx, final_lbl

In [45]:
folds, idx, lbls = make_folds(df, 5)

In [46]:
lbls

[[2,
  0,
  2,
  0,
  2,
  2,
  0,
  0,
  0,
  1,
  0,
  2,
  2,
  2,
  1,
  0,
  0,
  2,
  2,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  2,
  0,
  1,
  1,
  0,
  2,
  2,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  0,
  1,
  1,
  0],
 [0,
  2,
  0,
  0,
  0,
  0,
  0,
  2,
  0,
  1,
  0,
  1,
  2,
  0,
  1,
  1,
  2,
  2,
  0,
  2,
  2,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  0,
  0,
  1,
  1,
  0,
  0,
  2,
  2,
  2,
  2,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  2,
  2,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1],
 [0,
  2,
  2,
  0,
  1,
  0,
  0,
  2,
  1,
  2,
  2,
  1,
  2,
  1,
  2,
  2,
  0,
  0,
  0,
  2,
  0,
  2,
  1,
  0,
  2,
  0,
  0,
  0,
  2,
  0,
  1,
  0,
  2,
  0,
  0,
  2,
  2,
  2,
  2,
  0,
  2,
  1,
  1,
  1,
  1,
  0,
  0